# Efficient generation of dynamic pulses - Hahn Echo
This script implement a simulated Echo sequence, as described in the relative [blog post](https://blogs.zhinst.com/andrea/2021/09/08/efficient-generation-of-dynamic-pulses/)


Copyright (C) 2021 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

In [ ]:
#Define the parameters of the instrument
dataserver_host = 'localhost'    #Hostname or IP address of the dataserer
dev_hd = 'devNNNN'               #Device ID of the HDAWG
awg_core = 0                     #AWG Core (equal to output_channel//2)

In [ ]:
#Imports and initializations
from zhinst.ziPython import ziDAQServer
daq = ziDAQServer(dataserver_host, 8004, 6)

from hdawg_utils import HDAWG_Core
hdawg = HDAWG_Core(daq, dev_hd, awg_core)

In [ ]:
#Waveform paramaters
hdawg.constants.PULSE_WIDTH = 10e-9 #ns
hdawg.constants.SIGMAS_PULSE = 6

#Sequence parameters
hdawg.constants.T_START = 16
hdawg.constants.T_END = 10240
hdawg.constants.T_STEP = 27

#Readout parameters
hdawg.constants.TRIGGER_LEN = 32
hdawg.constants.READOUT_LEN = 1024

#Command Table paramaters
hdawg.constants.OFFSET_PI = 16

#amplitude paramaters
AMPLITUDE_PI = 1.0
AMPLITUDE_PI_2 = 0.5

#Creation of the command table
ct = []

#pi/2 entries
for i in range(16):
    entry = {
        "index": i,
        "waveform": {
            "index": i
        },
        "amplitude0": {
            "value": AMPLITUDE_PI_2,
            "increment": False
        }
    }
    ct.append(entry)

#pi entries
for i in range(16):
    entry = {
        "index": i+hdawg.constants.OFFSET_PI,
        "waveform": {
            "index": i
        },
        "amplitude0": {
            "value": AMPLITUDE_PI,
            "increment": False
        }
    }
    ct.append(entry)


seq_definition = """
//Calculations
const PULSE_WIDTH_SAMPLE = PULSE_WIDTH*DEVICE_SAMPLE_RATE;
const PULSE_TOTAL_LEN = ceil(PULSE_WIDTH_SAMPLE*SIGMAS_PULSE*2/16)*16;

//Waveform definition
wave pulse = gauss(PULSE_TOTAL_LEN, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);

//Create shifted waveforms
cvar j;
for (j = 0; j < 16; j++) {
  wave w_shifted = join(zeros(j), pulse, zeros(16-j));  //Create the j-samples shifted waveform
  assignWaveIndex(1,w_shifted, j);                      //Assign index j to the waveform
}


//Execution, no averaging
var t = T_START;
var tp;
var t_fine1, t_fine1_entry, coarse1;
var t2, t_fine2, coarse2;
do {
    //Calculate target tau (remove 16 for the right padding, common to all pulses)
    tp = t - 16;
    
    //The first delay doesn't need further adjustments, since the fine shift
    // of the first pulse is always zero, so we use t1 = tp
    t_fine1 = tp & 0xF;                   //Fine shift 1 (samples), four least significant bits
    t_fine1_entry = t_fine1 + OFFSET_PI;  //Fine shift 1 (CT entry index, adjust for pi-pulse)
    coarse1 = tp & -0x10;                 //Check if coarse delay 1 is needed (boolean)
    
    //Calculate target tau for the second wait.
    //Adjust for fine delay 1 by adding it to tp, so equal to
    // t2 = t - (16 - t_fine1)
    t2 = tp + t_fine1;
    t_fine2 = t2 & 0xF;                   //Fine shift 2 (samples and CT entry, no offset for pi/2-pulse)
    coarse2 = t2 & -0x10;                 //Check if coarse delay 2 is needed (boolean)
    
    resetOscPhase();                      //Reset the oscillator
    
    //First pi/2-pulse, no shift
    executeTableEntry(0);                 //Play first pulse, no fine shift
    
    //First tau wait time
    if(coarse1)
      playZero(tp);                       //Evolution time tau (coarse)
    
    //pi-pulse
    executeTableEntry(t_fine1_entry);     //Play second pulse, fine shift
    
    //Second tau wait time
    if(coarse2)
      playZero(t2);                       //Evolution time tau (coarse)
    
    //Second pi/2-pulse
    executeTableEntry(t_fine2);           //Play third pulse, fine shift

    
    playWave(marker(TRIGGER_LEN, 1));     //Readout trigger
    playZero(READOUT_LEN);                //Readout time

    t += T_STEP;                          //Increase wait time
} while (t < T_END);                      //Loop until the end
"""

hdawg.config(seq_definition, ct=ct)

In [ ]:
hdawg.run()